In [1]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
##  clases genericas
import numpy as np
import os
#import pickle
import pandas as pd
# import matplotlib.pyplot as plt

# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout, Dense, Input, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.models import Model

from sklearn.metrics import  confusion_matrix, roc_auc_score, f1_score,  precision_score, recall_score #, roc_curve, auc


from keras.preprocessing import image as keras_image
import keras

In [3]:
from tensorflow.keras.applications import *
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg19 import preprocess_input

from keras_efficientnets import *
from keras_efficientnets import preprocess_input  

from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.applications.nasnet import preprocess_input    
from tensorflow.keras.applications.xception import preprocess_input    
from tensorflow.keras.applications.inception_v3 import preprocess_input    
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input   

In [4]:
redes = ['DenseNet121', 'DenseNet169', 'DenseNet201', \
        'EfficientNetB0', 'EfficientNetB1', 'EfficientNetB2', 'EfficientNetB3', 'EfficientNetB4', \
        'InceptionV3', 'InceptionResNetV2', \
        'MobileNet', 'MobileNetV2', \
        'NASNetMobile', 'NASNetLarge', \
        'ResNet50', 'ResNet101', 'ResNet152', 'ResNet50V2', 'ResNet101V2', 'ResNet152V2', \
        'VGG16', 'VGG19', \
        'Xception']
len(redes)

23

### Dataset

In [7]:
### Definir paths em meu computador                 

Path_open_csv =  '/home/karem/Artigo_Periodico/1_Dataset/CSV_Files/'
Path_images =  '/home/karem/Artigo_Periodico/1_Dataset/1_Original_size640/'
#Path_images =  '/home/karem/Artigo_Periodico/1_Dataset/2_PreProcessing_size640/'
Path_save_result = '/home/karem/Artigo_Periodico/Classifier/Result/'  ## definir pasta onde salvar pesos e log
Path_Open_peso   =  '/home/karem/Artigo_Periodico/Classifier/Experiment/'  ## definir pasta onde salvar pesos e log

term_imagem = '.png'


###############################################################################################
#          Definicion de parametros de treinamento
################################################################################################

### Definir parâmetros
width       = 32*8*1
height      = 32*8*1
depth       = 3                 # 3 se RGB, 1 se grayscale (weights_ini= None)
inputShape  = (height, width, depth)


weights_ini = 'imagenet'  # 'imagenet'  None
NUM_EPOCHS = 80
BATCH_SIZE  = 8


################################################################################################
#                   carregar e montar csv de treino e validação
################################################################################################
### carregar e montar csv de treino e validação
data_test = pd.DataFrame()
data_test_temp = pd.read_csv(Path_open_csv + 'test.csv').rename(columns={'id': 'image_id', 'class_name': 'label'})
data_test_temp['label'].replace({'negative': 0, 'typical': 1, 'indeterminate': 1}, inplace=True)

data_test_temp['Image_name'] = data_test_temp['image_id'].replace('_image', '', regex=True) + term_imagem
data_test = data_test_temp[['Image_name', 'label']]

data_test['classVector'] = data_test['label']
data_test['Image_Index'] = Path_images + data_test['Image_name'] 

/home/karem/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/karem/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [8]:
def create_folders(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [9]:
class Data_GeneratorRx(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data_path_img, data_lab, index_b, batch_size=BATCH_SIZE, inputShape=inputShape, aug = 0, shuffle=True):            
        'Initialization'
        if aug == 1:
            data_gen = ImageDataGenerator(  
                                        rotation_range     = 15,
                                        # zoom_range         = 0.05,
                                        width_shift_range  = 0.1,
                                        height_shift_range = 0.1,
                                        # shear_range        = 0.01,
                                        horizontal_flip    = True,
                                        fill_mode          = "nearest",
                                        preprocessing_function = preprocess_input,
                                        )
        else:
            data_gen = ImageDataGenerator(
                                    preprocessing_function = preprocess_input, # se usar isto a imagem entrada deve ficar 0-255
                                 )            
        self.data_path_img  =  data_path_img   
        self.labels         =  data_lab   
        self.index_b        =  index_b    
        self.batch_size     =  batch_size # int
        self.inputShape     =  inputShape               # tuple int)
        self.data_gen       =  data_gen
        self.shuffle        =  shuffle
        self.aug            =  aug
        self.on_epoch_end()
        
####################################################################################################################################
    def __len__(self):
        """Denotes the number of batches per epoch
        :return: number of batches per epoch        """
        return int(np.ceil( len(self.data_path_img)  /  float(self.batch_size) ))
    
####################################################################################################################################
    def __getitem__(self, index):
        """Generate one batch of data
        :param index: index of the batch
        :return: X and y when fitting. X only when predicting
        """
        ## Generate indexes of the batch
        indexes = self.index_b[index * self.batch_size:(index + 1) * self.batch_size]
        ## Find list of IDs (paths)
        list_IDs_temp = [self.data_path_img[k] for k in indexes]
        ## Generate data
        X = self._generate_X(list_IDs_temp)
        y = self.labels[indexes]                
        ## Generate Augmented data
        iterat = self.data_gen.flow(X, y, shuffle=self.shuffle)
        X0,y0  = iterat.next()
        y_mat  = np.array(np.uint32 (y0.tolist()) )
        return X0,y_mat
    
####################################################################################################################################
    def _generate_X(self, list_IDs_temp):
        """Generates data containing batch_size images
        :param list_IDs_temp: list of image paths to load
        :return: batch of images
        """
        # Initialization
        X = np.empty( (len(list_IDs_temp), *self.inputShape) )
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            X[i,] = self._load_image(ID)
        return X
    
####################################################################################################################################
    def _load_image(self, image_path):
        """Load grayscale image
        :param image_path: path to image to load
        :return: loaded image
        """
        img = keras_image.load_img(image_path ,target_size=self.inputShape,interpolation='bicubic',color_mode = "rgb")
        img = 255.0*np.array(img) / (1.0*np.nanmax(img))
        return img   #  [0,255]
    
####################################################################################################################################
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.index_b = np.arange(len(self.data_path_img))
        if self.shuffle == True:
            np.random.shuffle(self.index_b)      

In [10]:
################################################################################################
#                                REDE
################################################################################################
    
def get_model(model_name, model_weights_ini, inputShape):        
    if model_name == 'ResNet50':          
        base_model = ResNet50(include_top=False,  weights = model_weights_ini, input_tensor=Input(shape=inputShape))
    
    elif model_name == 'ResNet101':          
        base_model = ResNet101(include_top=False,  weights = model_weights_ini, input_tensor=Input(shape=inputShape))
    
    if model_name == 'ResNet152':          
        base_model = ResNet152(include_top=False,  weights = model_weights_ini, input_tensor=Input(shape=inputShape))
    
    elif model_name   == 'ResNet50V2':                 
        base_model = ResNet50V2(include_top=False,  weights = model_weights_ini, input_tensor=Input(shape=inputShape))
    
    elif model_name   == 'ResNet101V2':                 
        base_model = ResNet101V2(include_top=False,  weights = model_weights_ini, input_tensor=Input(shape=inputShape))
    
    elif model_name   == 'ResNet152V2':                 
        base_model = ResNet152V2(include_top=False,  weights = model_weights_ini, input_tensor=Input(shape=inputShape))
    
    elif model_name == 'VGG16':          
        base_model = VGG16(include_top=False,  weights = model_weights_ini, input_tensor=Input(shape=inputShape))
    
    elif model_name == 'VGG19':            
        base_model = VGG19(include_top=False,  weights = model_weights_ini, input_tensor=Input(shape=inputShape))
        
    elif model_name == 'EfficientNetB0':  
        base_model = EfficientNetB0(include_top=False,  weights = model_weights_ini, input_tensor=Input(shape=inputShape))
    
    elif model_name == 'EfficientNetB1':  
        base_model = EfficientNetB1(include_top=False,  weights = model_weights_ini, input_tensor=Input(shape=inputShape))
    
    elif model_name == 'EfficientNetB2':  
        base_model = EfficientNetB2(include_top=False,  weights = model_weights_ini, input_tensor=Input(shape=inputShape))
    
    elif model_name == 'EfficientNetB3':  
        base_model = EfficientNetB3(include_top=False,  weights = model_weights_ini, input_tensor=Input(shape=inputShape))
    
    elif model_name == 'EfficientNetB4':        
        base_model = EfficientNetB4(include_top=False,  weights = model_weights_ini, input_tensor=Input(shape=inputShape))   
    
    elif model_name == 'EfficientNetB5':  
        base_model = EfficientNetB5(include_top=False,  weights = model_weights_ini, input_tensor=Input(shape=inputShape))
    
    elif model_name == 'EfficientNetB6':  
        base_model = EfficientNetB6(include_top=False,  weights = model_weights_ini, input_tensor=Input(shape=inputShape))
    
    elif model_name == 'EfficientNetB7':  
        base_model = EfficientNetB7(include_top=False,  weights = model_weights_ini, input_tensor=Input(shape=inputShape))
        
    elif model_name == 'MobileNetV2':             
        base_model = MobileNetV2(include_top=False,  weights = model_weights_ini, input_tensor=Input(shape=inputShape)) 
    
    elif model_name == 'MobileNet':             
        base_model = MobileNet(include_top=False,  weights = model_weights_ini, input_tensor=Input(shape=inputShape)) 
    
    elif model_name == 'DenseNet121':
        base_model = DenseNet121(include_top=False,  weights = model_weights_ini, input_tensor=Input(shape=inputShape))   
    
    elif model_name == 'DenseNet169':
        base_model = DenseNet169(include_top=False,  weights = model_weights_ini, input_tensor=Input(shape=inputShape))   
    
    elif model_name == 'DenseNet201':
        base_model = DenseNet201(include_top=False,  weights = model_weights_ini, input_tensor=Input(shape=inputShape))   
    
    elif model_name == 'NASNetLarge':
        base_model = NASNetLarge(include_top=False,  weights = model_weights_ini, input_tensor=Input(shape=inputShape)) 
    
    elif model_name == 'NASNetMobile':
        base_model = NASNetMobile(include_top=False,  weights = model_weights_ini, input_tensor=Input(shape=inputShape)) 
    
    elif model_name == 'Xception':             
        base_model = Xception(include_top=False,  weights = model_weights_ini, input_tensor=Input(shape=inputShape))   
        
    elif model_name == 'InceptionV3':             
        base_model = InceptionV3(include_top=False,  weights = model_weights_ini, input_tensor=Input(shape=inputShape))   
        
    elif model_name == 'InceptionResNetV2':             
        base_model = InceptionResNetV2(include_top=False,  weights = model_weights_ini, input_tensor=Input(shape=inputShape))   
    return base_model

In [11]:
def network(rede):
    
    ################################################################################################
    #                                Dados do treino 
    ################################################################################################

    l_nome_proceso = 'CNN_mlp'
    experimento = l_nome_proceso + '_WH_' + str(width) 

    ### definir pasta onde esta o peso treinado
    Path_Open_peso_bes   = Path_Open_peso + '/Models/' + experimento +  '/'

    ### Criar pastas onde estão os pesos
    #create_folders(Path_Open_peso)
    create_folders(Path_save_result)

    ### definir nomes de arquivos de saida
    palavra_s = 'ensaio_' + rede + '_' + weights_ini + '_WH_' + str(width)       
    arquivo_best_peso = os.path.sep.join([Path_Open_peso_bes, palavra_s + ".hdf5"]) 
    
    
    ### definir dictionario para apontador dentro do generator
    paramsB = {
                  'batch_size': BATCH_SIZE,
                  'inputShape': inputShape,
              } 
    
    
    ################################################################################################
    #                  Montar CONV net
    ################################################################################################
    ##  carregar modelo base
    base_model = get_model(rede, weights_ini, inputShape)
    ## construir MLP
    headModel = base_model.output
    headModel = GlobalAveragePooling2D()(headModel)
    headModel = BatchNormalization()(headModel)
    headModel = Dense(256, activation="relu")(headModel)
    headModel = Dropout(0.3)(headModel)                      # orig = 0.5
    headModel = Dense(1, activation="sigmoid", name="pred")(headModel)

    ## place the head FC model on top of the base model (this will become the actual model we will train)
    model = Model(inputs=base_model.input, outputs=headModel, name="Classifica")

    ## carregar pesos
    model.load_weights(arquivo_best_peso)
    
    
    ################################################################################################
    #                  Montar generator
    ################################################################################################  
    test_generator  =  Data_GeneratorRx(data_path_img = data_test['Image_Index']  , data_lab = data_test['classVector']  , index_b = np.arange(len(data_test)), **paramsB, aug = 0,shuffle = False)

    
    ################################################################################################
    #                  predecir
    ################################################################################################
    y_pred = model.predict(test_generator,    verbose=1  )
    y_true = np.array(np.uint32 (data_test['classVector'].tolist()) ).reshape(len(data_test['classVector']),1)

    ## Assumir  threshold
    threshold = [0.5]
    AUC     = roc_auc_score(y_true, y_pred)  

    ## Find prediction  applying threshold
    y_pred_bin = np.array([1 if prob >= threshold else 0 for prob in y_pred])
    y_pred_bin = y_pred_bin.reshape([len(y_pred_bin),1])

    ## Calcular metricas
    tn0, fp0, fn0, tp0 = confusion_matrix(y_true , y_pred_bin).ravel()
    f1    = f1_score(y_true, y_pred_bin, average='binary')
    acc   = (tp0 + tn0) / (tp0 + tn0 + fp0+ fn0)
    ps    = precision_score(y_true , y_pred_bin)
    rec   = recall_score(y_true , y_pred_bin)

    ## salvar metricas
    metricas = np.zeros([y_pred.shape[1],12])
    metricas[:] = [0,0, threshold[0],tp0, fp0, tn0, fn0, AUC, f1 , acc, ps, rec]
    salida   = pd.DataFrame(metricas).rename(columns={0:'rede', 1:'Base',  2:'threshold',3:'TP', 4:'FP', 5:'TN', 6:'FN', 7:'AUROC', 8:'F1', 9:'acc', 10:'prec', 11: 'rec'})
    salida['Base'] = 'exemplo cnn'
    salida['rede'] = rede

    #salida.to_csv(Path_save_result + 'res_' + rede + '_' + weights_ini + '_WH_' + str(width) + '.csv' , index = False)
    #salida.to_csv(Path_save_result + 'res_' + rede + '_' + weights_ini + '_WH_' + str(width) + '_ep_' + str(ep) + '.csv' , index = False)

    
    
    
    # Salva o resultado no arquivo csv
    #salida.to_csv(Path_save_result + 'res_' + rede + '_' + weights_ini + '_WH_' + str(width) + '.csv' , index = False)

    # Se o arquivo não existir, irá criar e salvar os dados da predição
    if not os.path.isfile(Path_save_result + 'result_' + weights_ini + '_WH_' + str(width) + '.csv'):
        salida.to_csv(Path_save_result + 'result_' + weights_ini + '_WH_' + str(width) + '.csv', index = False)    
    else: # caso contrário irá concatenar o que existe com os dados de outras redes
        df_1 = pd.read_csv(Path_save_result + 'result_' + weights_ini + '_WH_' + str(width) + '.csv') # dado existente no csv
        df_2 = salida # dado obtido na última predição

        result = pd.concat([df_1, df_2], axis = 0) # concatenate vertically (linhas) / se axis = 1 => concatenate horizontally (colunas)
        result = result.reset_index(drop = True) # reinicia o index
        result.to_csv(Path_save_result + 'result_' + weights_ini + '_WH_' + str(width) + '.csv', index = False)

In [12]:
for i in range(len(redes)):
    
    rede = redes[i]
    
    network(rede)    
    
    print('Fim da rede: ', rede)   

71/71 [==============================] - 69s 971ms/step
Fim da rede:  DenseNet121
71/71 [==============================] - 35s 500ms/step
Fim da rede:  DenseNet169
71/71 [==============================] - 33s 462ms/step
Fim da rede:  DenseNet201
71/71 [==============================] - 42s 587ms/step
Fim da rede:  EfficientNetB0
71/71 [==============================] - 14s 197ms/step
Fim da rede:  EfficientNetB1
71/71 [==============================] - 35s 488ms/step
Fim da rede:  EfficientNetB2
71/71 [==============================] - 46s 643ms/step
Fim da rede:  EfficientNetB3
71/71 [==============================] - 37s 521ms/step
Fim da rede:  EfficientNetB4
71/71 [==============================] - 60s 850ms/step
Fim da rede:  InceptionV3
71/71 [==============================] - 40s 564ms/step
Fim da rede:  InceptionResNetV2
71/71 [==============================] - 16s 223ms/step
Fim da rede:  MobileNet
71/71 [==============================] - 21s 300ms/step
Fim da rede:  MobileNet

In [13]:
dff = pd.read_csv(Path_save_result + 'result_' + weights_ini + '_WH_' + str(width) + '.csv') # dado existente no csv
dff

,rede,Base,threshold,TP,FP,TN,FN,AUROC,F1,acc,prec,rec
0,DenseNet121,exemplo cnn,0.5,357.0,69.0,106.0,32.0,0.886831,0.876074,0.820922,0.838028,0.917738
1,DenseNet169,exemplo cnn,0.5,344.0,30.0,145.0,45.0,0.927448,0.901704,0.867021,0.919786,0.884319
2,DenseNet201,exemplo cnn,0.5,341.0,60.0,115.0,48.0,0.895733,0.863291,0.808511,0.850374,0.876607
3,EfficientNetB0,exemplo cnn,0.5,304.0,32.0,143.0,85.0,0.878458,0.838621,0.792553,0.904762,0.781491
4,EfficientNetB1,exemplo cnn,0.5,347.0,56.0,119.0,42.0,0.909658,0.876263,0.826241,0.861042,0.892031
5,EfficientNetB2,exemplo cnn,0.5,332.0,38.0,137.0,57.0,0.905046,0.874835,0.831560,0.897297,0.853470
6,EfficientNetB3,exemplo cnn,0.5,343.0,44.0,131.0,46.0,0.894660,0.884021,0.840426,0.886305,0.881748
7,EfficientNetB4,exemplo cnn,0.5,312.0,25.0,150.0,77.0,0.904311,0.859504,0.819149,0.925816,0.802057
8,InceptionV3,exemplo cnn,0.5,359.0,65.0,110.0,30.0,0.913698,0.883149,0.831560,0.846698,0.922879
9,InceptionResNetV2,exemplo cnn,0.5,353.0,56.0,119.0,36.0,0.906206,0.884712,0.836879,0.863081,0.907455


In [11]:
dff = pd.read_csv(Path_save_result + 'result_' + weights_ini + '_WH_' + str(width) + '.csv') # dado existente no csv
dff

,rede,Base,threshold,TP,FP,TN,FN,AUROC,F1,acc,prec,rec
0,DenseNet121,exemplo cnn,0.5,323.0,36.0,139.0,66.0,0.910922,0.863636,0.819149,0.899721,0.830334
1,DenseNet169,exemplo cnn,0.5,349.0,48.0,127.0,40.0,0.921660,0.888041,0.843972,0.879093,0.897172
2,DenseNet201,exemplo cnn,0.5,364.0,70.0,105.0,25.0,0.910525,0.884569,0.831560,0.838710,0.935733
3,EfficientNetB0,exemplo cnn,0.5,344.0,46.0,129.0,45.0,0.906662,0.883184,0.838652,0.882051,0.884319
4,EfficientNetB1,exemplo cnn,0.5,340.0,46.0,129.0,49.0,0.913434,0.877419,0.831560,0.880829,0.874036
5,EfficientNetB2,exemplo cnn,0.5,344.0,53.0,122.0,45.0,0.889357,0.875318,0.826241,0.866499,0.884319
6,EfficientNetB3,exemplo cnn,0.5,335.0,62.0,113.0,54.0,0.875725,0.852417,0.794326,0.843829,0.861183
7,EfficientNetB4,exemplo cnn,0.5,320.0,34.0,141.0,69.0,0.893426,0.861373,0.817376,0.903955,0.822622
8,InceptionV3,exemplo cnn,0.5,365.0,70.0,105.0,24.0,0.910518,0.885922,0.833333,0.839080,0.938303
9,InceptionResNetV2,exemplo cnn,0.5,337.0,49.0,126.0,52.0,0.893044,0.869677,0.820922,0.873057,0.866324


In [12]:
dff = pd.read_csv(Path_save_result + 'result_' + weights_ini + '_WH_' + str(width) + '.csv') # dado existente no csv
dff

,rede,Base,threshold,TP,FP,TN,FN,AUROC,F1,acc,prec,rec
0,DenseNet121,exemplo cnn,0.5,320.0,27.0,148.0,69.0,0.902152,0.869565,0.829787,0.922190,0.822622
1,DenseNet169,exemplo cnn,0.5,358.0,78.0,97.0,31.0,0.876710,0.867879,0.806738,0.821101,0.920308
2,DenseNet201,exemplo cnn,0.5,318.0,31.0,144.0,71.0,0.907029,0.861789,0.819149,0.911175,0.817481
3,EfficientNetB0,exemplo cnn,0.5,369.0,149.0,26.0,20.0,0.745017,0.813671,0.700355,0.712355,0.948586
4,EfficientNetB1,exemplo cnn,0.5,386.0,160.0,15.0,3.0,0.725641,0.825668,0.710993,0.706960,0.992288
5,EfficientNetB2,exemplo cnn,0.5,371.0,128.0,47.0,18.0,0.852075,0.835586,0.741135,0.743487,0.953728
6,EfficientNetB3,exemplo cnn,0.5,389.0,169.0,6.0,0.0,0.740624,0.821542,0.700355,0.697133,1.000000
7,EfficientNetB4,exemplo cnn,0.5,387.0,174.0,1.0,2.0,0.746339,0.814737,0.687943,0.689840,0.994859
8,InceptionV3,exemplo cnn,0.5,343.0,49.0,126.0,46.0,0.908527,0.878361,0.831560,0.875000,0.881748
9,InceptionResNetV2,exemplo cnn,0.5,317.0,28.0,147.0,72.0,0.900081,0.863760,0.822695,0.918841,0.814910


In [13]:
dff = pd.read_csv('/home/karem/Artigo_Periodico/Classifier/Result_/' + 'result_' + weights_ini + '_WH_' + str(width) + '.csv') # dado existente no csv
dff

,rede,Base,threshold,TP,FP,TN,FN,AUROC,F1,acc,prec,rec
0,DenseNet201,exemplo cnn,0.5,340.0,35.0,140.0,49.0,0.918061,0.890052,0.851064,0.906667,0.874036
1,MobileNetV2,exemplo cnn,0.5,349.0,56.0,119.0,40.0,0.893309,0.879093,0.829787,0.861728,0.897172
2,ResNet50V2,exemplo cnn,0.5,351.0,46.0,129.0,38.0,0.911186,0.893130,0.851064,0.884131,0.902314
3,VGG16,exemplo cnn,0.5,360.0,61.0,114.0,29.0,0.912303,0.888889,0.840426,0.855107,0.925450
4,VGG19,exemplo cnn,0.5,349.0,50.0,125.0,40.0,0.904987,0.885787,0.840426,0.874687,0.897172
5,InceptionV3,exemplo cnn,0.5,343.0,49.0,126.0,46.0,0.908527,0.878361,0.831560,0.875000,0.881748
6,InceptionResNetV2,exemplo cnn,0.5,317.0,28.0,147.0,72.0,0.900081,0.863760,0.822695,0.918841,0.814910
7,Xception,exemplo cnn,0.5,332.0,41.0,134.0,57.0,0.904444,0.871391,0.826241,0.890080,0.853470
8,EfficientNetB0,exemplo cnn,0.5,330.0,49.0,126.0,59.0,0.877121,0.859375,0.808511,0.870712,0.848329
9,EfficientNetB4,exemplo cnn,0.5,350.0,45.0,130.0,39.0,0.925479,0.892857,0.851064,0.886076,0.899743
